In [1]:
import gymnasium
import push_box
import pybullet as p
import pybullet_data
import time
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os
from typing import Any, Dict
import torch as th

pybullet build time: Nov 28 2023 23:48:36


In [2]:
# p.disconnect()

In [3]:
log_path = os.path.join('Training', 'Logs')

In [4]:
env = gymnasium.make('pushBox-v0')
env = DummyVecEnv([lambda: env])

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce GTX 1080/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 545.23.06
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 545.23.06
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce GTX 1080/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [5]:
# Add a callback to training stage for early stopping
save_path = os.path.join('Training', 'SavedModels', 'PPO_65_best')
stop_callback = StopTrainingOnRewardThreshold(reward_threshold = 100, verbose = 1)
eval_callback = EvalCallback(env, 
                            callback_on_new_best = stop_callback,
                            eval_freq = 10000, 
                            best_model_save_path = save_path, 
                            verbose = 1)


In [6]:
# Learning rate schedule: linearly decreasing from 0.0007 to 0.0001
def linear_lr(progress_remaining: float):
    start_lr = 0.0005
    end_lr = 0.0001
    return end_lr + (start_lr - end_lr) * progress_remaining

In [7]:
model = PPO('MlpPolicy', env, learning_rate=linear_lr, verbose=1, tensorboard_log=log_path)

Using cuda device


/home/danielmasamba/anaconda3/envs/dan/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [8]:
# model.learn(total_timesteps=20000000, callback=eval_callback)

In [9]:
PPO_Path = os.path.join('Training', 'SavedModels', 'PPO_65_20M_360_cont_actions')

In [10]:
# model.save(PPO_Path)

In [11]:
del model

In [12]:
model = PPO.load(PPO_Path, env=env)

In [13]:
evaluate_policy(model, env, n_eval_episodes=10)

/home/danielmasamba/anaconda3/envs/dan/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


box reached target


(0.5863447468913364, 1.6481599240623734)

In [14]:
# p.disconnect()